In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
import seaborn as sns

import glob, os, yaml, subprocess, itertools
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from statsmodels.distributions.empirical_distribution import ECDF
import sklearn.metrics
from sklearn.decomposition import PCA
import timeit
import scipy.stats as st
import statsmodels
import pickle, yaml, tracemalloc
from scipy.stats import binomtest

analysis_dir = '/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue'
input_data_dir = '/n/data1/hms/dbmi/farhat/ye12/who'
who_variants_combined = pd.read_csv("who_confidence_2021.csv")

import itertools
from stats_utils import *
import warnings
warnings.filterwarnings("ignore")

In [39]:
res = {'fit_time': np.array([0.46307302, 0.46552658, 0.46767092, 0.45148492, 0.46610522]), 
       'score_time': np.array([0.01107359, 0.00999832, 0.01002049, 0.00967932, 0.01317215]), 
       'test_roc_auc': np.array([0.97973397, 0.98943888, 0.95101111, 0.95274882, 0.96620066]), 
       'test_recall': np.array([0.95521739, 0.95523685, 0.89217391, 0.90434783, 0.93217391]), 
       'test_accuracy': np.array([0.97737876, 0.97879259, 0.94937783, 0.95220588, 0.95616516]), 
       'test_balanced_accuracy': np.array([0.97163761, 0.97269387, 0.93456139, 0.93981012, 0.94995116])
      }

res['test_spec'] = 2 * res['test_balanced_accuracy'] - res['test_recall']

In [36]:
res['test_roc_auc'].mean()

0.967826688

In [40]:
res['test_recall'].mean()

0.9278299779999999

In [41]:
res['test_spec'].mean()

0.9796316820000002

In [42]:
res['test_accuracy'].mean()

0.962784044

In [43]:
res['test_balanced_accuracy'].mean()

0.9537308300000001

# RIFAMPICIN TEST CASE

# 0. Original Analysis: 137 new significant resistance-associated variants

## But, there are 152 with OR > 1 and BH p-val < 0.01 that are in tier 2 genes (rpoB is the only tier 1)
## These 152 mutations were studied in the next two analyses

In [154]:
folder = "BINARY"
phenos_name = "WHO"
drug = "Rifampicin"
drug_WHO_abbr = "RIF"

# get the number of high confidence resistance-associated 
tiers1_stats = pd.read_csv(os.path.join(analysis_dir, drug, "BINARY/tiers=1/phenos=WHO/dropAF_noSyn_unpooled", "model_analysis_with_stats.csv"))
tiers12_stats = pd.read_csv(os.path.join(analysis_dir, drug, "BINARY/tiers=1+2/phenos=WHO/dropAF_noSyn_unpooled", "model_analysis_with_stats.csv"))

In [155]:
tiers12_stats.query("mutation=='rpoC_p.Glu1092Asp'")[['mutation', 'predicted_effect', 'position', 'confidence', 'Odds_Ratio',
       'pval', 'BH_pval', 'Bonferroni_pval', 'Num_Isolates', 'TP', 'FP', 'TN', 'FN', 'PPV', 'Sens', 'Spec', 'PPV_LB', 'PPV_UB']]

,mutation,predicted_effect,position,confidence,Odds_Ratio,pval,BH_pval,Bonferroni_pval,Num_Isolates,TP,FP,TN,FN,PPV,Sens,Spec,PPV_LB,PPV_UB
16,rpoC_p.Glu1092Asp,missense_variant,766645,5) Not assoc w R,1.216332,0.0,0.0,0.0,2697,1711,986,19935,8352,0.634409,0.170029,0.95287,0.615909,0.652616


In [156]:
1711/(1711+986)

0.6344086021505376

# 1. Likelihood Ratio Test: 5/152 of the Above Tier 2 Mutations are Significant

152 tier 2 variants were found significant in the first analysis. We then performed a likelihood ratio test:

A mutation is removed from the original model input matrix, and then a new model is fit. 

We then compare the log-likelihoods of the original model and the model with 1 less mutation. If the p-value is significant, it means that removing the mutation significantly changes the fit of the model. 

In [2]:
drug = "Rifampicin"
phenos_name = "WHO"
out_dir = os.path.join(analysis_dir, drug, f"BINARY/LRT/phenos={phenos_name}")

LRTresults_tier1 = pd.read_csv(os.path.join(out_dir, f"results_tier={'1'}.csv")).rename(columns={"Unnamed: 0": "mutation"})
LRTresults_tier2 = pd.read_csv(os.path.join(out_dir, f"results_tier={'2'}.csv")).rename(columns={"Unnamed: 0": "mutation"})

# remove the FULL model row, which is the first row
LRTresults_tier1 = add_pval_corrections(LRTresults_tier1.iloc[1:, ])
LRTresults_tier2 = add_pval_corrections(LRTresults_tier2.iloc[1:, ])

tier1_model_permute = pd.read_csv(os.path.join(analysis_dir, drug, f"BINARY/tiers=1/phenos={phenos_name}/dropAF_noSyn_unpooled/model_analysis.csv")).query("~mutation.str.contains('PC')")
tier2_model_permute = pd.read_csv(os.path.join(analysis_dir, drug, f"BINARY/tiers=1+2/phenos={phenos_name}/dropAF_noSyn_unpooled/model_analysis.csv")).query("~mutation.str.contains('PC')")

# remove the tier 1 genes for the purposes of this analysis
tier2_model_permute = tier2_model_permute.query("mutation not in @tier1_model_permute.mutation")

# check that all mutations in the permutation dataframe are in the LRT dataframe. The only difference should be the FULL model row
# print(set(tier1_model_permute.mutation).symmetric_difference(LRTresults_tier1.index))
# print(set(tier2_model_permute.mutation).symmetric_difference(LRTresults_tier2.index))
assert len(set(tier1_model_permute.mutation).symmetric_difference(LRTresults_tier1.mutation)) == 0
assert len(set(tier2_model_permute.mutation).symmetric_difference(LRTresults_tier2.mutation)) == 0

# combine results into a single dataframe for easy searching
tier1_model = tier1_model_permute.merge(LRTresults_tier1[["mutation", "pval", "BH_pval", "Bonferroni_pval"]].rename(columns={"pval": "LRT_pval", 
                                                                                                               "BH_pval": "LRT_BH_pval", 
                                                                                                               "Bonferroni_pval": "LRT_Bonferroni_pval"
                                                                                                              }), on="mutation", how="left")

tier2_model = tier2_model_permute.merge(LRTresults_tier2[["mutation", "pval", "BH_pval", "Bonferroni_pval"]].rename(columns={"pval": "LRT_pval", 
                                                                                                               "BH_pval": "LRT_BH_pval", 
                                                                                                               "Bonferroni_pval": "LRT_Bonferroni_pval"
                                                                                                              }), on="mutation", how="left")

print(len(tier1_model), len(tier2_model))

815 2845


In [4]:
# mutations picked up by the LRT, but not the coefficient association. 
# They are not significant in the tiers 1+2 model either 
thresh = 0.05

print(f"{len(tier1_model.query('Odds_Ratio > 1 & BH_pval < @thresh & LRT_BH_pval > @thresh'))} significant coef, but not LRT")
print(f"{len(tier1_model.query('Odds_Ratio > 1 & BH_pval > @thresh & LRT_BH_pval < @thresh'))} significant LRT, but not coef")
print(f"{len(tier1_model.query('Odds_Ratio > 1 & BH_pval < @thresh & LRT_BH_pval < @thresh'))} significant in both")

78 significant coef, but not LRT
2 significant LRT, but not coef
51 significant in both


In [7]:
thresh = 0.01
tier2_model.query('Odds_Ratio > 1 & (BH_pval < @thresh & LRT_BH_pval < @thresh)')

,mutation,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,Odds_Ratio,OR_LB,OR_UB,confidence,LRT_pval,LRT_BH_pval,LRT_Bonferroni_pval
2,rpoC_p.Glu1092Asp,0.195840,0.128329,0.264205,0.0,0.0,0.0,1.216332,1.136927,1.302395,5) Not assoc w R,5.033098e-07,0.000716,0.001432
19,Rv2752c_p.Asn30Ser,0.099958,0.039851,0.149232,0.0,0.0,0.0,1.105124,1.040655,1.160942,3) Uncertain significance,1.480226e-07,0.000421,0.000421


In [9]:
# this was the other mutation they called attention to as not being resistance-associated
tier2_model.query("mutation=='glpK_p.Val192fs'")

,mutation,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,Odds_Ratio,OR_LB,OR_UB,confidence,LRT_pval,LRT_BH_pval,LRT_Bonferroni_pval
203,glpK_p.Val192fs,0.026607,-0.011454,0.062963,0.032,0.255294,1.0,1.026964,0.988611,1.064987,NaN,1.0,1.0,1.0


In [10]:
thresh = 0.01

print(f"{len(tier2_model.query('Odds_Ratio > 1 & BH_pval < @thresh & LRT_BH_pval > @thresh'))} significant coef, but not LRT")
print(f"{len(tier2_model.query('Odds_Ratio > 1 & BH_pval > @thresh & LRT_BH_pval < @thresh'))} significant LRT, but not coef")
print(f"{len(tier2_model.query('Odds_Ratio > 1 & BH_pval < @thresh & LRT_BH_pval < @thresh'))} significant in both")

101 significant coef, but not LRT
0 significant LRT, but not coef
2 significant in both


In [11]:
thresh = 0.05
tier1_sig_variants = tier1_model.query('Odds_Ratio > 1 & (BH_pval < @thresh | LRT_BH_pval < @thresh)')["mutation"].values

thresh = 0.01
tier2_sig_variants = tier2_model.query('Odds_Ratio > 1 & (BH_pval < @thresh | LRT_BH_pval < @thresh)')["mutation"].values
print(len(tier1_sig_variants), len(tier2_sig_variants))

combined_variants = np.concatenate([tier1_sig_variants, tier2_sig_variants])
print(len(np.unique(combined_variants)))
        
combined_variants_df = pd.DataFrame({"mutation": combined_variants, "Tier": ["1"]*len(tier1_sig_variants) + ["2"]*len(tier2_sig_variants)})

# USE THIS LIST FOR THE FINAL PREDICTIVE MODELS AND TO MEASURE THE AUC INCREASE
combined_variants_df.to_csv(os.path.join(analysis_dir, drug, "BINARY", "significant_tiers=1+2_variants.csv"), index=False)

131 103
234


# Perform bootstrapping for all 152 mutations to see how much AUC, Sens, Spec, and Accuracy Change

If the confidence interval lies above 0, it means that removing the feature SIGNIFICANTLY DECREASES THE PREDICTIVE POWER OF THE MODEL

CI > 0 = FEATURE IS IMPORTANT

In [127]:
# # bootstrapped the difference: full model stat - (model - 1) stat
# LRT_bootstrap_StatsDiff = pd.read_csv(os.path.join(analysis_dir, drug, "BINARY/LRT", f"{phenos_name}phenos_bootstrap_StatsDiff.csv.gz"), compression="gzip")
# print(len(LRT_bootstrap_StatsDiff.mutation.unique()))
# LRT_bootstrap_StatsDiff2 = pd.read_csv(os.path.join(analysis_dir, drug, "BINARY/LRT", f"{phenos_name}phenos_bootstrap_StatsDiff_2.csv.gz"), compression="gzip")
# print(len(LRT_bootstrap_StatsDiff2.mutation.unique()))

# LRT_bootstrap_StatsDiff = pd.concat([LRT_bootstrap_StatsDiff, LRT_bootstrap_StatsDiff2])

tier1_metrics_CV = pd.read_csv(os.path.join(out_dir, f"tier=1_binaryMetrics_CV.csv"), index_col=[0])
# tier1_metrics_CV = tier1_metrics_CV.iloc[:, :-1]
# tier1_metrics_CV.columns = ["AUC", "Sens", "Spec", "Acc"]
ref_tier1 = tier1_metrics_CV.loc["FULL"]

tier2_metrics_CV = pd.read_csv(os.path.join(out_dir, f"tiers=1+2_binaryMetrics_CV.csv"), index_col=[0])
# tier2_metrics_CV = tier2_metrics_CV.iloc[:, :-1]
# tier2_metrics_CV.columns = ["AUC", "Sens", "Spec", "Acc"]
ref_tier2 = tier2_metrics_CV.loc["FULL"]

In [ ]:
# summary_stats_all = pd.read_csv(os.path.join(out_dir, f"tiers=1+2_binaryMetrics_CV.csv"))
# drug_WHO_abbr = "RIF"

# summary_stats_all = summary_stats_all.merge(who_variants_combined.query("drug==@drug_WHO_abbr")[["mutation", "confidence"]],
#                        on="mutation", how="left"
#                       )

# summary_stats_all.to_csv(os.path.join(out_dir, f"tier=1_binaryMetrics_CV.csv"), index=False)

In [152]:
thresh = 0.05
for mutation in tier1_metrics_CV.index.unique():
        
    # can also use alternative = 'greater' to see if removing the mutation causes a significant decrease in metrics
    pvals = st.ttest_ind(ref_tier1.iloc[:, :-1], tier1_metrics_CV.loc[mutation].iloc[:, :-1], axis=0, equal_var=False, alternative='greater')[1]
        
    # any p-value is less than the threshold, print it
    if sum(pvals < thresh) > 0:
        print(f"{mutation}, {tier1_metrics_CV.loc[mutation, 'confidence'].unique()[0]} \n{dict(zip(tier1_metrics_CV.columns, np.round(pvals, 4)))}")
        
        

rpoB_p.Asp435Val, 1) Assoc w R 
{'test_roc_auc': 0.0528, 'test_sens': 0.0079, 'test_spec': 0.5104, 'test_accuracy': 0.0389, 'test_balanced_accuracy': 0.0185}
rpoB_p.Ser450Leu, 1) Assoc w R 
{'test_roc_auc': 0.001, 'test_sens': 0.0004, 'test_spec': 0.0064, 'test_accuracy': 0.0024, 'test_balanced_accuracy': 0.0014}


In [153]:
thresh = 0.01
for mutation in tier2_metrics_CV.index.unique():
        
    # can also use alternative = 'greater' to see if removing the mutation causes a significant decrease in metrics
    pvals = st.ttest_ind(ref_tier2, tier2_metrics_CV.loc[mutation], axis=0, equal_var=False, alternative='greater')[1]
        
    # any p-value is less than the threshold, print it
    if sum(pvals < thresh) > 0:
        print(f"{mutation}, {tier2_metrics_CV.loc[mutation, 'confidence'].unique()[0]} \n{dict(zip(tier2_metrics_CV.columns, np.round(pvals, 4)))}")


# Model with Interaction Terms

Chose 2 variants because they were the only Tier 2 genes that were significantly associated in the Ridge regression and were significant in the LRT

In [60]:
thresh = 0.01
tier2_model.query('BH_pval < @thresh & LRT_BH_pval < @thresh')

,mutation,coef,pval,BH_pval,Bonferroni_pval,Odds_Ratio,confidence,LRT_pval,LRT_BH_pval,LRT_Bonferroni_pval
2,rpoC_p.Glu1092Asp,0.195840,0.0,0.0,0.0,1.216332,5) Not assoc w R,5.033098e-07,0.000716,0.001432
19,Rv2752c_p.Asn30Ser,0.099958,0.0,0.0,0.0,1.105124,3) Uncertain significance,1.480226e-07,0.000421,0.000421


In [61]:
thresh = 0.01
interact_mut = tier2_model.query('BH_pval < @thresh & LRT_BH_pval < @thresh')["mutation"].values
df_phenos = pd.read_csv(os.path.join(analysis_dir, drug, "phenos_binary.csv")).set_index("sample_id")
#interact_mut = ['rpoC_p.Glu1092Asp', 'Rv2752c_p.Asn30Ser']

# model_matrix = pd.read_pickle(os.path.join(analysis_dir, drug, "BINARY/tiers=1+2/phenos=WHO/dropAF_noSyn_unpooled/model_matrix.pkl"))
# interact_matrix = model_matrix.copy()

# # get a list of all pairs of mutations to interact
# interactions = list(itertools.product(*[interact_mut, interact_matrix.columns[interact_matrix.columns.str.contains("rpoB")]]))

# # add interactions
# for (mut1, mut2) in interactions:
#     interact_matrix[f"{mut1}+{mut2}"] = interact_matrix[mut1] * interact_matrix[mut2]
    
# # drop any columns with no signal
# interact_matrix = interact_matrix[interact_matrix.columns[~((interact_matrix == 0).all())]]

# assert len(set(model_matrix.columns) - set(interact_matrix.columns)) == 0
# print(len(interact_matrix), len(model_matrix))

# interact_matrix.to_pickle(os.path.join(analysis_dir, drug, f"BINARY/interaction/model_matrix_{phenos_name}.pkl"))

interact_matrix = pd.read_pickle(os.path.join(analysis_dir, drug, f"BINARY/interaction/model_matrix_{phenos_name}.pkl"))

interact_matrix = interact_matrix.merge(df_phenos, left_index=True, right_index=True, how="left")
del interact_matrix["phenotypic_category"]

interact_matrix.shape

(30984, 3737)

In [62]:
# so much co-occurrence of rpoC_p.Glu1092Asp with rpoB mutations. Can also check proportion that are high confidence rpoB mutations
for mut in interact_mut:
    print(mut, interact_matrix[interact_matrix.columns[interact_matrix.columns.str.contains(mut)]].shape[1])

rpoC_p.Glu1092Asp 125
Rv2752c_p.Asn30Ser 4


In [63]:
phenos_name = "WHO"
interact_res = pd.read_csv(os.path.join(analysis_dir, drug, f"BINARY/interaction/{phenos_name}_coef.csv"))
interact_permute = pd.read_csv(os.path.join(analysis_dir, drug, f"BINARY/interaction/{phenos_name}_coef_permutation.csv"))

# assess significance using the results of the permutation test
for i, row in interact_res.iterrows():
    # p-value is the proportion of permutation coefficients that are AT LEAST AS EXTREME as the test statistic
    if row["coef"] > 0:
        interact_res.loc[i, "pval"] = np.mean(interact_permute[row["mutation"]] >= row["coef"])
    else:
        interact_res.loc[i, "pval"] = np.mean(interact_permute[row["mutation"]] <= row["coef"])
        
interact_res = add_pval_corrections(interact_res)
interact_res["Odds_Ratio"] = np.exp(interact_res["coef"])

interact_res[["Mut1", "Mut2"]] = interact_res["mutation"].str.split("+", expand=True)
interact_res = interact_res.merge(who_variants_combined.query("drug=='RIF'"), left_on="Mut2", right_on="mutation", how="left")
del interact_res["mutation_y"]
del interact_res["drug"]
interact_res.rename(columns={"mutation_x": "mutation"}, inplace=True)

In [68]:
interact_res.query("mutation.str.contains('rpoC_p.Glu1092Asp') & BH_pval < 0.05").sort_values(["BH_pval", "Odds_Ratio"], 
                                                                                           ascending=[True, False]
                                                                                          )

,mutation,coef,pval,BH_pval,Bonferroni_pval,Odds_Ratio,Mut1,Mut2,confidence
3206,rpoC_p.Glu1092Asp,0.219309,0.000,0.000000,0.0,1.245216,rpoC_p.Glu1092Asp,None,NaN
3676,rpoC_p.Glu1092Asp+rpoB_p.Ile491Tyr,0.026609,0.000,0.000000,0.0,1.026966,rpoC_p.Glu1092Asp,rpoB_p.Ile491Tyr,3) Uncertain significance
3693,rpoC_p.Glu1092Asp+rpoB_p.Met434_Asn437delinsIle,0.018817,0.000,0.000000,0.0,1.018995,rpoC_p.Glu1092Asp,rpoB_p.Met434_Asn437delinsIle,2) Assoc w R - Interim
3623,rpoC_p.Glu1092Asp+rpoB_p.Asn260Asp,-0.027974,0.000,0.000000,0.0,0.972414,rpoC_p.Glu1092Asp,rpoB_p.Asn260Asp,NaN
3725,rpoC_p.Glu1092Asp+rpoB_p.Val170Phe,-0.057892,0.003,0.038700,1.0,0.943752,rpoC_p.Glu1092Asp,rpoB_p.Val170Phe,1) Assoc w R
3708,rpoC_p.Glu1092Asp+rpoB_p.Ser450Leu,0.057540,0.004,0.048584,1.0,1.059228,rpoC_p.Glu1092Asp,rpoB_p.Ser450Leu,1) Assoc w R


In [70]:
interact_res.query("mutation.str.contains('rpoB_p.Val170Phe')")

,mutation,coef,pval,BH_pval,Bonferroni_pval,Odds_Ratio,Mut1,Mut2,confidence
2936,rpoB_p.Val170Phe,0.191617,0.000,0.0000,0.0,1.211206,rpoB_p.Val170Phe,None,NaN
3725,rpoC_p.Glu1092Asp+rpoB_p.Val170Phe,-0.057892,0.003,0.0387,1.0,0.943752,rpoC_p.Glu1092Asp,rpoB_p.Val170Phe,1) Assoc w R


In [67]:
interact_res.query("mutation.str.contains('rpoB_p.Asn260Asp')")

,mutation,coef,pval,BH_pval,Bonferroni_pval,Odds_Ratio,Mut1,Mut2,confidence
2395,rpoB_p.Asn260Asp,-0.027974,0.0,0.0,0.0,0.972414,rpoB_p.Asn260Asp,None,NaN
3623,rpoC_p.Glu1092Asp+rpoB_p.Asn260Asp,-0.027974,0.0,0.0,0.0,0.972414,rpoC_p.Glu1092Asp,rpoB_p.Asn260Asp,NaN


In [217]:
interact_matrix.groupby("phenotype")['rpoC_p.Glu1092Asp+rpoB_p.Ser450Leu'].value_counts()

phenotype  rpoC_p.Glu1092Asp+rpoB_p.Ser450Leu
0          0.0                                   20907
           1.0                                      14
1          0.0                                    8682
           1.0                                    1381
Name: rpoC_p.Glu1092Asp+rpoB_p.Ser450Leu, dtype: int64

In [224]:
interact_matrix['rpoC_p.Glu1092Asp+rpoB_p.Ser450Leu'].mean()

0.045023237800154915

In [218]:
1381 / 1385

0.9971119133574007

In [223]:
interact_matrix['rpoC_p.Glu1092Asp'].mean()

0.08704492641363284

In [222]:
interact_matrix['rpoB_p.Ser450Leu'].mean()

0.21449780531887425

In [34]:
os.path.join()

'/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue/Rifampicin/BINARY/tiers=1/phenos=WHO/dropAF_noSyn_unpooled'